In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
#data = yf.download(tickers = '^RUI', start = '2012-03-11',end = '2022-07-10')
data = yf.download(tickers='BTC-usd', priod= 'max',interval='1d')
#data.head(10)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-01-03,16688.847656,16760.447266,16622.371094,16679.857422,16679.857422,13903079207
2023-01-04,16680.205078,16964.585938,16667.763672,16863.238281,16863.238281,18421743322
2023-01-05,16863.472656,16884.021484,16790.283203,16836.736328,16836.736328,13692758566


In [1]:
# Adding indicators
data['RSI']=ta.rsi(data.Close, length=15)
data['EMAF']=ta.ema(data.Close, length=20)
data['EMAM']=ta.ema(data.Close, length=100)
data['EMAS']=ta.ema(data.Close, length=150)

data['Target'] = data['Adj Close']-data.Open
data['Target'] = data['Target'].shift(-1)

data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)

data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

NameError: name 'ta' is not defined

In [ ]:
data_set = data.iloc[:, 0:11]#.values
pd.set_option('display.max_columns', None)

#data_set.head(20)
data_set
#print(data_set.shape)
#print(data.shape)
#print(type(data_set))

In [4]:
#Target column Categories
#y =[1 if data.Open[i]>data.Close[i] else 0 for i in range(0, len(data))]
#yi = [data.Open[i]-data.Close[i] for i in range(0, len(data))]
#print(yi)
#print(len(yi))

In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(-1,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[-0.99964654 -0.99952087 -0.99934439 ...  0.02003178  1.
  -0.9986096 ]
 [-0.99924383 -0.99895064 -0.99891328 ...  0.0140514  -1.
  -0.99927757]
 [-0.99859105 -0.99878137 -0.99915032 ...  0.01696932 -1.
  -0.99930673]
 ...
 [-0.50586515 -0.51472239 -0.4996959  ...  0.02752984  1.
  -0.50385334]
 [-0.50602125 -0.51298484 -0.49796991 ... -0.01007656 -1.
  -0.50987003]
 [-0.50373312 -0.51182617 -0.50311619 ... -0.01158037 -1.
  -0.5162095 ]]


In [6]:
# multiple feature from data provided to the model
X = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 180
print(data_set_scaled.shape[0])
for j in range(8):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X)
print(X.shape)
print(y)
print(y.shape)

2875
[[[-0.99964654 -0.99952087 -0.99934439 ... -1.         -0.99829104
   -0.99697281]
  [-0.99924383 -0.99895064 -0.99891328 ... -0.99991529 -0.9983137
   -0.99700873]
  [-0.99859105 -0.99878137 -0.99915032 ... -0.99990669 -0.99835219
   -0.99705562]
  ...
  [-0.99848387 -0.99874077 -0.9981596  ... -0.99843037 -0.99920992
   -0.99937771]
  [-0.99835431 -0.99873992 -0.9980953  ... -0.99847353 -0.99920921
   -0.99937814]
  [-0.99838805 -0.99864209 -0.99805003 ... -0.99849468 -0.99920424
   -0.99937555]]

 [[-0.99924383 -0.99895064 -0.99891328 ... -0.99991529 -0.9983137
   -0.99700873]
  [-0.99859105 -0.99878137 -0.99915032 ... -0.99990669 -0.99835219
   -0.99705562]
  [-0.99926471 -0.9995424  -0.99910989 ... -0.99990187 -0.99839063
   -0.99710238]
  ...
  [-0.99835431 -0.99873992 -0.9980953  ... -0.99847353 -0.99920921
   -0.99937814]
  [-0.99838805 -0.99864209 -0.99805003 ... -0.99849468 -0.99920424
   -0.99937555]
  [-0.99820225 -0.99863372 -0.99800848 ... -0.99852595 -0.99920227
   

In [7]:
#also comprehensions for X
#X = np.array([data_set_scaled[i-backcandles:i,:4].copy() for i in range(backcandles,len(data_set_scaled))])
#print(X)
#print(X.shape)

In [8]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train.shape)

2156
(2156, 180, 8)
(539, 180, 8)
(2156, 1)
(539, 1)
(2156, 1)


In [9]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(100, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss="mean_squared_error")
print(model.summary())
model.fit(x=X_train, y=y_train, batch_size=32, epochs=50, shuffle=True, validation_split = 0.1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 180, 8)]          0         
                                                                 
 first_layer (LSTM)          (None, 100)               43600     
                                                                 
 dense_layer (Dense)         (None, 1)                 101       
                                                                 
 output (Activation)         (None, 1)                 0         
                                                                 
Total params: 43,701
Trainable params: 43,701
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
61/61 [==============================] - 24s 231ms/step - loss: 0.0948 - val_loss: 0.0470
Epoch 2/50
61/61 [==============================] - 12s 194ms/step - loss: 0.0016 -

In [10]:
y_pred = model.predict(X_test)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
   print(y_pred[i], y_test[i])
  

17/17 [==============================] - 2s 76ms/step
[-0.06674043] [-0.03002807]
[-0.0691288] [-0.00269165]
[-0.07954112] [-0.01093118]
[-0.08071733] [0.0104378]
[-0.08294454] [-0.02175932]
[-0.0782081] [-0.03523458]
[-0.08314992] [-0.03166188]
[-0.09129575] [-0.06259043]
[-0.09735923] [-0.07325584]
[-0.11034627] [-0.06994422]


In [11]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape
print("the MAPE is : ", MAPE(y_test,y_pred))

    

the MAPE is :  52.129596374886155


In [1]:
plt.figure(figsize=(8,8))
plt.plot(y_test, color = 'blue', label = 'Test')
plt.plot(y_pred, color = 'red', label = 'pred')
plt.legend()
plt.show()

NameError: name 'plt' is not defined